DATASET Link:

https://www.kaggle.com/datasets/atakanak/turkish-house-rent-prediction-dataset/data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [2]:
dataset_train = pd.read_csv("Dataset_TR/train.csv")
dataset_test = pd.read_csv("Dataset_TR/test.csv")

In [3]:
dataset_train.shape

(6166, 16)

In [4]:
dataset_test.shape

(2642, 16)

In [5]:
dataset_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,6166.0,3082.500000,1780.115212,0.0,1541.25,3082.5,4623.75,6165.0
Bathroom Count,6164.0,1.389358,0.791561,1.0,1.00,1.0,2.00,22.0
Price,6166.0,71509.498540,783516.108690,1.0,14000.00,20000.0,30000.00,40000000.0


In [6]:
dataset_train.head()

,Unnamed: 0,City,Town,Neighborhood,Apartment Type,House Type,House Age,House Size,Room Count,Floor,Furniture,Bathroom Count,Hand,Heater Type,Heater Fuel,Price
0,0,Ankara,Mamak,Akşemsettin,Daire,Daire,3,125 m2,3 + 1,Ara Kat,True,2.0,İkinci El,Kombi,Doğalgaz,16000
1,1,Adana,Seyhan,Yeşilyurt,Daire,Daire,5,140 m2,3 + 1,13. Kat,False,2.0,İkinci El,Kombi,Doğalgaz,15500
2,2,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,Sıfır,360 m2,4 + 1,21 ve üzeri,False,3.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000
3,3,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,Sıfır,350 m2,4 + 1,Ara Kat,False,4.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000
4,4,İstanbul,Şişli,Fulya,Daire,Daire,15,90 m2,1 + 1,5. Kat,True,1.0,İkinci El,Merkezi (Pay Ölçer),Doğalgaz,60000


In [7]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6166 entries, 0 to 6165
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      6166 non-null   int64  
 1   City            6166 non-null   object 
 2   Town            6166 non-null   object 
 3   Neighborhood    6166 non-null   object 
 4   Apartment Type  6166 non-null   object 
 5   House Type      6166 non-null   object 
 6   House Age       6166 non-null   object 
 7   House Size      6166 non-null   object 
 8   Room Count      6166 non-null   object 
 9   Floor           5760 non-null   object 
 10  Furniture       5934 non-null   object 
 11  Bathroom Count  6164 non-null   float64
 12  Hand            5043 non-null   object 
 13  Heater Type     6166 non-null   object 
 14  Heater Fuel     5147 non-null   object 
 15  Price           6166 non-null   int64  
dtypes: float64(1), int64(2), object(13)
memory usage: 770.9+ KB


In [8]:
dataset_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,6166.0,3082.500000,1780.115212,0.0,1541.25,3082.5,4623.75,6165.0
Bathroom Count,6164.0,1.389358,0.791561,1.0,1.00,1.0,2.00,22.0
Price,6166.0,71509.498540,783516.108690,1.0,14000.00,20000.0,30000.00,40000000.0


In [9]:
dataset_train.isnull().sum()

Unnamed: 0           0
City                 0
Town                 0
Neighborhood         0
Apartment Type       0
House Type           0
House Age            0
House Size           0
Room Count           0
Floor              406
Furniture          232
Bathroom Count       2
Hand              1123
Heater Type          0
Heater Fuel       1019
Price                0
dtype: int64

In [10]:
dataset_train.columns

Index(['Unnamed: 0', 'City', 'Town', 'Neighborhood', 'Apartment Type',
       'House Type', 'House Age', 'House Size', 'Room Count', 'Floor',
       'Furniture', 'Bathroom Count', 'Hand', 'Heater Type', 'Heater Fuel',
       'Price'],
      dtype='object')

In [11]:
for column in dataset_train.columns:
    print(column)
    print(pd.unique(dataset_train[column]))
    print("\n====================\n")

Unnamed: 0
[   0    1    2 ... 6163 6164 6165]


City
['Ankara' 'Adana' 'İstanbul' 'Muğla' 'Antalya' 'İzmir' 'Isparta' 'Aksaray'
 'Balıkesir' 'Tekirdağ' 'Edirne' 'Tokat' 'Bursa' 'Kocaeli' 'Aydın'
 'Erzincan' 'Burdur' 'Çanakkale' 'Yalova' 'Zonguldak' 'Kayseri' 'Giresun'
 'Eskişehir' 'Bolu' 'Düzce' 'Kastamonu' 'Amasya' 'Kahramanmaraş' 'Çorum'
 'Mersin(İçel)' 'Erzurum' 'Hatay' 'Samsun' 'Denizli' 'Manisa' 'Batman'
 'Bartın' 'Ordu' 'Gaziantep' 'Sakarya' 'Sivas' 'Kırıkkale' 'Kütahya'
 'Uşak' 'Nevşehir' 'Niğde' 'Osmaniye' 'Sinop' 'Rize' 'Trabzon' 'Konya'
 'Diyarbakır' 'Kırklareli' 'Karaman' 'Elazığ' 'Bilecik' 'Karabük' 'Mardin'
 'Van' 'Yozgat' 'Muş' 'Afyonkarahisar' 'K.K.T.C.' 'Kilis' 'Şanlıurfa'
 'Artvin' 'Malatya' 'Kırşehir' 'Adıyaman']


Town
['Mamak' 'Seyhan' 'Beşiktaş' 'Şişli' 'Bodrum' 'Alanya' 'Bahçelievler'
 'Seferihisar' 'Çeşme' 'Serik' 'Etimesgut' 'Ortaca' 'Kadıköy' 'Sarıyer'
 'Çankaya' 'Beyoğlu' 'Avcılar' 'Merkez' 'Küçükçekmece' 'Erdek' 'Keçiören'
 'Kağıthane' 'Beykoz' 'Zeytinburnu'

In [12]:
pd.unique(dataset_train["City"])

array(['Ankara', 'Adana', 'İstanbul', 'Muğla', 'Antalya', 'İzmir',
       'Isparta', 'Aksaray', 'Balıkesir', 'Tekirdağ', 'Edirne', 'Tokat',
       'Bursa', 'Kocaeli', 'Aydın', 'Erzincan', 'Burdur', 'Çanakkale',
       'Yalova', 'Zonguldak', 'Kayseri', 'Giresun', 'Eskişehir', 'Bolu',
       'Düzce', 'Kastamonu', 'Amasya', 'Kahramanmaraş', 'Çorum',
       'Mersin(İçel)', 'Erzurum', 'Hatay', 'Samsun', 'Denizli', 'Manisa',
       'Batman', 'Bartın', 'Ordu', 'Gaziantep', 'Sakarya', 'Sivas',
       'Kırıkkale', 'Kütahya', 'Uşak', 'Nevşehir', 'Niğde', 'Osmaniye',
       'Sinop', 'Rize', 'Trabzon', 'Konya', 'Diyarbakır', 'Kırklareli',
       'Karaman', 'Elazığ', 'Bilecik', 'Karabük', 'Mardin', 'Van',
       'Yozgat', 'Muş', 'Afyonkarahisar', 'K.K.T.C.', 'Kilis',
       'Şanlıurfa', 'Artvin', 'Malatya', 'Kırşehir', 'Adıyaman'],
      dtype=object)

## Fix Room Count

In [13]:
pd.unique(dataset_train["Room Count"])

array(['3 + 1', '4 + 1', '1 + 1', '2 + 1', '5 + 1', '5 + 2', '7 + 1',
       '6 + 1', '8 + 1', '6 + 2', '10 + 2', '8 + 2', '3 + 2', '7 + 2',
       '4 + 2', '1 + 0', '2 + 0', '2 + 2', '6 + 3', '11 + 2', '9 + 3',
       '6 + 6', '6 + 0', '9 + 1', '25 + 2', '4 + 8', '1 + 3', '3 + 3',
       '3 + 0', '4 + 4', '5 + 3', '1 + 2', '41 + 1', '9 + 2', '10 + 1'],
      dtype=object)

In [14]:
room_counts = dataset_train["Room Count"].value_counts()
room_counts

3 + 1     2170
2 + 1     2086
1 + 1     1058
4 + 1      428
5 + 1      119
1 + 0       57
5 + 2       49
4 + 2       45
3 + 2       36
6 + 1       28
2 + 0       19
6 + 2       14
2 + 2       10
7 + 1       10
7 + 2        6
3 + 3        3
8 + 1        3
9 + 2        2
6 + 3        2
1 + 2        2
6 + 6        2
6 + 0        2
5 + 3        2
8 + 2        2
41 + 1       1
4 + 4        1
3 + 0        1
25 + 2       1
1 + 3        1
4 + 8        1
9 + 1        1
9 + 3        1
11 + 2       1
10 + 2       1
10 + 1       1
Name: Room Count, dtype: int64

In [15]:
valid_rooms = room_counts[room_counts >= 10].index
filtered_df = dataset_train[dataset_train["Room Count"].isin(valid_rooms)]

In [16]:
filtered_df.shape

(6129, 16)

In [17]:
filtered_df["Room Count"].value_counts()

3 + 1    2170
2 + 1    2086
1 + 1    1058
4 + 1     428
5 + 1     119
1 + 0      57
5 + 2      49
4 + 2      45
3 + 2      36
6 + 1      28
2 + 0      19
6 + 2      14
7 + 1      10
2 + 2      10
Name: Room Count, dtype: int64

In [18]:
filtered_df[["Rooms", "Living Rooms"]] = filtered_df["Room Count"].str.split(r"\s*\+\s*", expand=True)

c:\Users\Giray\Projects\Rent_Estimation_System\tensorflow_env\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
filtered_df.shape

(6129, 18)

In [20]:
filtered_df[["Room Count","Rooms","Living Rooms"]]

,Room Count,Rooms,Living Rooms
0,3 + 1,3,1
1,3 + 1,3,1
2,4 + 1,4,1
3,4 + 1,4,1
4,1 + 1,1,1
...,...,...,...
6161,3 + 1,3,1
6162,3 + 1,3,1
6163,3 + 1,3,1
6164,3 + 1,3,1


In [21]:
filtered_df["Rooms"] = filtered_df["Rooms"].astype(int)
filtered_df["Living Rooms"] = filtered_df["Living Rooms"].astype(int)
filtered_df.drop(columns=["Room Count"], inplace=True)

c:\Users\Giray\Projects\Rent_Estimation_System\tensorflow_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\Giray\Projects\Rent_Estimation_System\tensorflow_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Giray\Projects\Rent_Estimation_System\tensorflow_env\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set o

In [22]:
filtered_df.shape

(6129, 17)

In [23]:
filtered_df

,Unnamed: 0,City,Town,Neighborhood,Apartment Type,House Type,House Age,House Size,Floor,Furniture,Bathroom Count,Hand,Heater Type,Heater Fuel,Price,Rooms,Living Rooms
0,0,Ankara,Mamak,Akşemsettin,Daire,Daire,3,125 m2,Ara Kat,True,2.0,İkinci El,Kombi,Doğalgaz,16000,3,1
1,1,Adana,Seyhan,Yeşilyurt,Daire,Daire,5,140 m2,13. Kat,False,2.0,İkinci El,Kombi,Doğalgaz,15500,3,1
2,2,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,Sıfır,360 m2,21 ve üzeri,False,3.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000,4,1
3,3,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,Sıfır,350 m2,Ara Kat,False,4.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000,4,1
4,4,İstanbul,Şişli,Fulya,Daire,Daire,15,90 m2,5. Kat,True,1.0,İkinci El,Merkezi (Pay Ölçer),Doğalgaz,60000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6161,6161,İzmir,Çeşme,Boyalık,Villa,Villa,11,250 m2,NaN,True,2.0,İkinci El,Klima,Elektrik,650000,3,1
6162,6162,İstanbul,Kadıköy,Feneryolu,Daire,Daire,30,126 m2,8. Kat,False,1.0,NaN,Merkezi,Doğalgaz,30000,3,1
6163,6163,İstanbul,Bahçelievler,Siyavuşpaşa,Daire,Daire,30,125 m2,4. Kat,False,1.0,İkinci El,Kombi,Doğalgaz,18000,3,1
6164,6164,İstanbul,Şişli,Esentepe,Daire,Daire,13,120 m2,2. Kat,False,2.0,İkinci El,Kombi,Doğalgaz,40000,3,1


## Analyze House Age

In [24]:
pd.unique(filtered_df["House Age"])

array(['3', '5', 'Sıfır', '15', '10', '21', '1', '27', '25', '11', '22',
       '2', '4', '30', '26', '24', '55', '35', '9', '14', '40', '31',
       '20', '8', '6', '18', '12', '7', '39', '13', '45', '17', '50',
       '16', '36', '100', '75', '19', '43', '28', '37', '33', '38', '69',
       '42', '32', '29', '60', '34', '23', '48', '70', '52', '200', '80',
       '220', '95', '46', '65', '49'], dtype=object)

In [25]:
filtered_df["House Age"].value_counts()

Sıfır    686
25       430
5        424
30       414
15       359
20       347
10       345
35       230
4        229
1        194
2        172
8        170
3        156
11       146
7        144
40       139
12       123
6        121
26       119
16       118
31        95
18        89
21        83
9         77
28        74
13        64
22        64
14        61
45        51
17        47
27        43
24        34
50        32
23        32
38        27
42        23
29        22
32        22
19        20
33        20
36        18
34        16
55        10
100        8
39         6
60         4
37         3
43         3
75         2
48         2
70         2
69         1
52         1
200        1
80         1
220        1
95         1
46         1
65         1
49         1
Name: House Age, dtype: int64

In [26]:
filtered_df.groupby("House Age").agg(
    Mean_Price=("Price", "mean"),
    House_Count=("Price", "count")
).sort_values("House_Count", ascending=False)


,Mean_Price,House_Count
House Age,,
Sıfır,62053.810496,686
25,31779.418605,430
5,51853.077830,424
30,115888.647343,414
15,43504.568245,359
20,161986.942363,347
10,58609.417391,345
35,30815.217391,230
4,79758.515284,229


In [27]:
def clean_house_age(value):
    if value == "Sıfır":
        return 0
    try:
        return int(value)
    except:
        return None  # veya np.nan

filtered_df["House Age"] = filtered_df["House Age"].apply(clean_house_age)


c:\Users\Giray\Projects\Rent_Estimation_System\tensorflow_env\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [28]:
filtered_df.groupby("House Age").agg(
    Mean_Price=("Price", "mean"),
    House_Count=("Price", "count")
).sort_values("House_Count", ascending=False)

,Mean_Price,House_Count
House Age,,
0,62053.810496,686
25,31779.418605,430
5,51853.077830,424
30,115888.647343,414
15,43504.568245,359
20,161986.942363,347
10,58609.417391,345
35,30815.217391,230
4,79758.515284,229


In [29]:
age_counts = filtered_df["House Age"].value_counts()
valid_ages = age_counts[age_counts >= 10].index

filtered_df = filtered_df[filtered_df["House Age"].isin(valid_ages)]

In [30]:
filtered_df.groupby("House Age").agg(
    Mean_Price=("Price", "mean"),
    House_Count=("Price", "count")
).sort_values("House_Count", ascending=False)

,Mean_Price,House_Count
House Age,,
0,62053.810496,686
25,31779.418605,430
5,51853.077830,424
30,115888.647343,414
15,43504.568245,359
20,161986.942363,347
10,58609.417391,345
35,30815.217391,230
4,79758.515284,229


## Analyze House Size

In [31]:
filtered_df["House Size"].value_counts()

110 m2      354
100 m2      349
120 m2      346
90 m2       323
130 m2      313
           ... 
720 m2        1
539 m2        1
282 m2        1
1.620 m2      1
510 m2        1
Name: House Size, Length: 239, dtype: int64

In [32]:
non_numeric_sizes = filtered_df[~filtered_df["House Size"].str.contains(r"\d", na=False)]


In [33]:
print("Non-numeric Data count:", len(non_numeric_sizes)) # Removing rows like "Kat Karşılığı :"

Non-numeric Data count: 5


In [34]:
filtered_df = filtered_df[filtered_df["House Size"].str.contains(r"\d", na=False)]


In [35]:
non_numeric_sizes = filtered_df[~filtered_df["House Size"].str.contains(r"\d", na=False)]
print("Non-numeric Data count:", len(non_numeric_sizes))

Non-numeric Data count: 0


In [36]:
def extract_m2(value):
    try:
        return float(value.replace(" m2", "").replace(".", ""))
    except:
        return None

filtered_df["House Size m2"] = filtered_df["House Size"].apply(extract_m2)
filtered_df.drop(columns=["House Size"], inplace=True)


In [37]:
filtered_df.shape

(6085, 17)

In [38]:
filtered_df.head()

,Unnamed: 0,City,Town,Neighborhood,Apartment Type,House Type,House Age,Floor,Furniture,Bathroom Count,Hand,Heater Type,Heater Fuel,Price,Rooms,Living Rooms,House Size m2
0,0,Ankara,Mamak,Akşemsettin,Daire,Daire,3,Ara Kat,True,2.0,İkinci El,Kombi,Doğalgaz,16000,3,1,125.0
1,1,Adana,Seyhan,Yeşilyurt,Daire,Daire,5,13. Kat,False,2.0,İkinci El,Kombi,Doğalgaz,15500,3,1,140.0
2,2,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,0,21 ve üzeri,False,3.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000,4,1,360.0
3,3,İstanbul,Beşiktaş,Nisbetiye,Residence,Residence,0,Ara Kat,False,4.0,Sıfır,Merkezi (Pay Ölçer),Doğalgaz,350000,4,1,350.0
4,4,İstanbul,Şişli,Fulya,Daire,Daire,15,5. Kat,True,1.0,İkinci El,Merkezi (Pay Ölçer),Doğalgaz,60000,1,1,90.0


## Analyze Floor

In [39]:
filtered_df["Floor"].value_counts(dropna=False)

2. Kat             852
3. Kat             712
1. Kat             682
Ara Kat            519
4. Kat             485
Yüksek Giriş       411
NaN                381
5. Kat             289
En Üst Kat         251
Bahçe Katı         250
Giriş Katı         204
6. Kat             124
7. Kat             115
Kot 1              110
8. Kat             101
21 ve üzeri         77
Zemin               63
9. Kat              55
Çatı Katı           54
10. Kat             47
11. Kat             37
Kot 2               31
Teras Katı          30
12. Kat             29
14. Kat             27
Villa Katı          26
13. Kat             26
Yarı Bodrum         21
15. Kat             14
Kot 3               14
18. Kat             11
17. Kat              9
16. Kat              9
19. Kat              7
20. Kat              5
Bodrum               4
Bodrum ve Zemin      2
Tripleks             1
Name: Floor, dtype: int64

In [40]:
def convert_floor(value):
    try:
        # ("2. Kat" → 2)
        if ". Kat" in value:
            return int(value.replace(". Kat", "").strip())
        
        # Special Floors:
        elif value == "Giriş Katı":
            return 0
        elif value == "Zemin":
            return 0
        elif value == "Bahçe Katı":
            return -1
        elif value == "Yüksek Giriş":
            return 0.5
        elif value == "Ara Kat":
            return None  # Uncertain, can be any floor between (0,top). But there are sufficient amount of data. Giving NaN for to use in Model.
        elif value == "En Üst Kat":
            return 21
        elif value == "Çatı Katı":
            return 21
        elif value == "Teras Katı":
            return 21
        elif value == "Bodrum":
            return -2
        elif value == "Yarı Bodrum":
            return -1.5
        elif "Kot" in value:
            return -1 * int(value.replace("Kot ", "").strip())
        elif "ve üzeri" in value:
            return 21
        else:
            return None  # Unsufficient amount of data: Tripleks, Villa Katı, etc.
    
    except:
        return None


In [41]:
filtered_df["Floor Level"] = filtered_df["Floor"].apply(convert_floor)

In [42]:
filtered_df["Floor Level"].value_counts(dropna=False)

 NaN     929
 2.0     852
 3.0     712
 1.0     682
 4.0     485
 21.0    412
 0.5     411
-1.0     360
 5.0     289
 0.0     267
 6.0     124
 7.0     115
 8.0     101
 9.0      55
 10.0     47
 11.0     37
-2.0      35
 12.0     29
 14.0     27
 13.0     26
-1.5      21
 15.0     14
-3.0      14
 18.0     11
 16.0      9
 17.0      9
 19.0      7
 20.0      5
Name: Floor Level, dtype: int64